In [1]:
from imputationLibrary import forwardFilling, hotDeck, meanImputation, movingAverage, splineInterpolation, separate, randomSampleImputation
from imputationLibrary.util import util
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from tsmoothie.smoother import *
import matplotlib.pyplot as plt

In [2]:
INPUT = 'output/\CompleteWeeklyIndexes'

In [3]:
df = pd.read_csv(INPUT+'.csv', parse_dates=True, index_col=0)
df_test = df.loc['2015-01-02':] #Selecting test data
df_train = df.loc['2000-01-01':'2015-01-01'] #Selecting training data

In [4]:
# Subsample para weekly

# Generating weekly sampled dataset
def generate_weekly(df, initial_friday):
    df_friday = initial_friday #datetime(2000,1,7)

    df_weekly = pd.DataFrame(columns = df.columns)
    
    while df_friday <= df.tail(1).index:
        df_weekly = df_weekly.append(df.loc[df_friday])
        df_friday+=timedelta(days=7)
    return df_weekly

In [6]:
for i in range(0,30):
    df_white_noise, df_seasonal, df_trended, df_trend_and_seasonal = separate.separate(df_train)
    df_seasonal_train, df_seasonal_test = randomSampleImputation.input(df.loc['2000-01-01':'2015-01-01', df_seasonal.columns], df.loc['2015-01-02':, df_seasonal.columns], sliding_window_size = 0.03)
    df_white_noise_train, df_white_noise_test =  meanImputation.input(df.loc['2000-01-01':'2015-01-01', df_white_noise.columns], df.loc['2015-01-02':, df_white_noise.columns])
    df_trended_train, df_trended_test =  forwardFilling.input(df.loc['2000-01-01':'2015-01-01', df_trended.columns], df.loc['2015-01-02':, df_trended.columns])
    df_trend_and_seasonal_train, df_trend_and_seasonal_test =  splineInterpolation.input(df.loc['2000-01-01':'2015-01-01', df_trend_and_seasonal.columns], df.loc['2015-01-02':, df_trend_and_seasonal.columns])
    df_train = pd.concat([df_seasonal_train, df_white_noise_train, df_trended_train, df_trend_and_seasonal_train], axis=1, join='outer')
    df_seasonal_test.index = df_white_noise_test.index
    df_test = pd.concat([df_seasonal_test, df_white_noise_test, df_trended_test, df_trend_and_seasonal_test], axis=1, join='outer')

    ### USE KALMAN FILTER TO SMOOTH ALL DATA (ONLY VISUALIZATION PURPOSE) ###

    smoother_train = KalmanSmoother(component='level_longseason', 
                              component_noise={'level':0.1, 'longseason':0.1}, 
                              n_longseasons=365)
    smoother_train.smooth(df_train.T)

    smoother_test = KalmanSmoother(component='level_longseason', 
                              component_noise={'level':0.1, 'longseason':0.1}, 
                              n_longseasons=365)
    smoother_test.smooth(df_test.T)

    df_train = pd.DataFrame(data = smoother_train.smooth_data.T, index = df_train.index, columns= df_train.columns)
    df_test = pd.DataFrame(data = smoother_test.smooth_data.T, index = df_test.index, columns= df_test.columns)

    df_train_weekly = generate_weekly(df_train, datetime(2000,1,7))
    df_test_weekly = generate_weekly(df_test, datetime(2015,1,2))

    df_train_weekly.to_csv(r'output\/CompleteIndexesWeeklyTrainNatureKalman_'+str(i)+'.csv', index = True)
    df_test_weekly.to_csv(r'output\/CompleteIndexesWeeklyTestNatureKalman_'+str(i)+'.csv', index = True)

C:\Users\silva\Anaconda3\envs\mestrado\lib\site-packages\numpy\core\fromnumeric.py:58: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)
C:\Users\silva\Anaconda3\envs\mestrado\lib\site-packages\numpy\core\fromnumeric.py:58: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)
C:\Users\silva\Anaconda3\envs\mestrado\lib\site-packages\numpy\core\fromnumeric.py:58: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)
C:\Users\silva\Anaconda3\envs\mestrado\lib\site-packages\numpy\core\fromnumeric.py:58: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)
C:\Users\silva\Anaconda3\env